In [1]:
from premise import *
from premise import __version__
import pandas as pd

In [2]:
clear_inventory_cache()

Inventory cache cleared!


In [2]:
import bw2data
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [3]:
ndb = NewDatabase(
    scenarios=[
        {"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
        {"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
        {"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
        {"model": "remind", "pathway": "SSP2-rollBack", "year": 2025,},
        {"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
        {"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
        {"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2025,},
        {"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
        {"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},
        {"model": "remind", "pathway": "SSP3-NDC", "year": 2025,},
        {"model": "remind", "pathway": "SSP3-NPi", "year": 2025,},
        {"model": "remind", "pathway": "SSP3-PkBudg1000", "year": 2025,},

        {"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
        {"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2025,},
        {"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

        {"model": "image", "pathway": "SSP1-L", "year": 2025,},
        {"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
        {"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
        {"model": "image", "pathway": "SSP2-L", "year": 2025,},
        {"model": "image", "pathway": "SSP2-M", "year": 2025,},
        {"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
        {"model": "image", "pathway": "SSP3-H", "year": 2025,},
        {"model": "image", "pathway": "SSP5-H", "year": 2025,},

        {"model": "tiam-ucl", "pathway": "SSP2-Base", "year": 2025,},
        {"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
        {"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
        {"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},
    ],
    source_db="ecoinvent-3.11-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.11", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="biosphere3",
    keep_imports_uncertainty=True
)

premise v.(2, 3, 0, 'dev1')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [4]:
import pandas as pd

# Load the Excel file to check all sheet names
file_path = 'mapping_overview.xlsx'
excel_file = pd.ExcelFile(file_path)

# Display all sheet names
sheet_names = excel_file.sheet_names

# Load data from each sheet into a dictionary of dataframes
dfs = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheet_names}

# Concatenate all the dataframes into one, adding a column to indicate the source sheet
combined_df = pd.concat([df.assign(Sheet=sheet) for sheet, df in dfs.items()], ignore_index=True)


In [5]:
combined_df = combined_df.loc[combined_df["Variable Type"]=="Production volume"]

In [6]:
len(combined_df)

1700

In [7]:
large_data = pd.DataFrame()
for scenario in ndb.scenarios:
    model = scenario["model"]
    pathway = scenario["pathway"]
    print(model, pathway)
    for i, row in combined_df.loc[combined_df["IAM Model"]==model].groupby(
        ["Sheet", "Variable"]
    ):

        var, _, _, _, sector = row.values.tolist()[0]

        data = None

        if var in (
            "CO2", "gdp", "population", "GMST"
        ):
            data = scenario["iam data"].other_vars.sel(variables=var)
        else:
            if var in scenario["iam data"].production_volumes.coords["variables"].values.tolist():
                data = scenario["iam data"].production_volumes.sel(variables=var)
            else:
                continue

        if data is not None:
            long_data = data.to_dataframe("val").reset_index()
            long_data["sector"] = sector
            long_data["model"] = model
            long_data["scenario"] = pathway
            long_data = long_data.loc[long_data["year"] <= 2100]
            large_data = pd.concat([large_data, long_data])

remind SSP1-NPi
remind SSP1-PkBudg650
remind SSP1-PkBudg1000
remind SSP2-rollBack
remind SSP2-NDC
remind SSP2-NPi
remind SSP2-PkBudg650
remind SSP2-PkBudg1000
remind SSP3-rollBack
remind SSP3-NDC
remind SSP3-NPi
remind SSP3-PkBudg1000
remind-eu SSP2-NDC
remind-eu SSP2-NPi
remind-eu SSP2-PkBudg650
remind-eu SSP2-PkBudg1000
image SSP1-L
image SSP1-Ma
image SSP1-VLLO
image SSP2-L
image SSP2-M
image SSP2-VLHO
image SSP3-H
image SSP5-H
tiam-ucl SSP2-Base
tiam-ucl SSP2-RCP19
tiam-ucl SSP2-RCP26
tiam-ucl SSP2-RCP45


In [8]:
sectors = large_data["sector"].unique().tolist()
for sector in sectors:
    print(sector)


Biomass
Carbon Dioxide Removal
Cement
Electricity
Fuels
Heat
Other
Steel
Transport Bus
Transport Passenger Cars
Transport Rail Freight
Transport Road Freight
Transport Sea Freight
Transport Two Wheelers


In [9]:
len(large_data)

1373932

In [ ]:
large_data.loc[large_data["variables"]=="population", "sector"] = "Population"
large_data.loc[large_data["variables"]=="CO2", "sector"] = "Carbon Dioxide emissions"
large_data.loc[large_data["variables"]=="gdp", "sector"] = "Gross Domestic Product"
large_data.loc[large_data["variables"]=="GMST", "sector"] = "GMST increase"

large_data.loc[
    (large_data["variables"].str.contains("diesel"))
    &(~large_data["variables"].str.contains("truck"))
    &(~large_data["variables"].str.contains("train"))
    &(~large_data["variables"].str.contains("bus"))
    &(~large_data["variables"].str.contains("car"))
    &(~large_data["variables"].str.contains("ship"))
    , "sector"] = "Diesel"

large_data.loc[large_data["variables"].str.contains("ethanol"), "sector"] = "Gasoline"
large_data.loc[
    (large_data["variables"].str.contains("gasoline"))
    &(~large_data["variables"].str.contains("truck"))
    &(~large_data["variables"].str.contains("train"))
    &(~large_data["variables"].str.contains("bus"))
    &(~large_data["variables"].str.contains("car"))
    &(~large_data["variables"].str.contains("two-wheeler"))
    , "sector"] = "Gasoline"

large_data.loc[large_data["variables"].str.contains("liquefied"), "sector"] = "LPG"
large_data.loc[large_data["variables"].str.contains("kerosene"), "sector"] = "Kerosene"
large_data.loc[large_data["variables"].str.contains("hydrogen"), "sector"] = "Hydrogen"

large_data.loc[large_data["variables"]=="natural gas", "sector"] = "Gas"
large_data.loc[large_data["variables"]=="biomethane", "sector"] = "Gas"
large_data.loc[large_data["variables"]=="heavy fuel oil", "sector"] = "Oil"

large_data = large_data.replace("truck, ", "", regex=True)
large_data = large_data.replace("train, ", "", regex=True)
large_data = large_data.replace("bus, ", "", regex=True)
large_data = large_data.replace("passenger car, ", "", regex=True)
large_data = large_data.replace("two-wheeler, ", "", regex=True)

large_data = large_data.replace(", mini", "", regex=True)
large_data = large_data.replace(", medium SUV", "", regex=True)
large_data = large_data.replace(", large SUV", "", regex=True)
large_data = large_data.replace(", medium", "", regex=True)
large_data = large_data.replace(", van", "", regex=True)
large_data = large_data.replace(", large", "", regex=True)
large_data = large_data.replace(", small", "", regex=True)

large_data = large_data.replace(", 3.5t", "", regex=True)
large_data = large_data.replace(", 7.5t", "", regex=True)
large_data = large_data.replace(", 18t", "", regex=True)
large_data = large_data.replace(", 26t", "", regex=True)
large_data = large_data.replace(", 40t", "", regex=True)

large_data = large_data.replace(", energy allocation", "", regex=True)
large_data = large_data.replace("kerosene, ", "", regex=True)
large_data = large_data.replace(", PEM", "", regex=True)
large_data = large_data.replace("hydrogen, ", "", regex=True)
large_data = large_data.replace("liquefied petroleum gas, ", "LPG", regex=True)
large_data = large_data.replace("steel - ", "", regex=True)

large_data = large_data.groupby([
    "region", "year", "variables", "sector", "model", "scenario"
]).sum().reset_index()


In [17]:
large_data.loc[
    (large_data["model"].isin(["image", "remind", "remind-eu"]))
    &(large_data["variables"]=="CO2"),
    "val"
] *= 1000000
large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"]=="CO2"),
    "val"
] *= 1000
large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"]=="gdp"),
    "val"
] *= 10000

large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["variables"].isin(['biomass crops - purpose grown', 'biomass wood - purpose grown', 'biomass - residual'])),
    "val"
] /= 1000

large_data.loc[
    (large_data["model"]=="remind")
    &(large_data["variables"].isin(['biomass crops - purpose grown', 'biomass wood - purpose grown', 'biomass - residual'])),
    "val"
] *= 10

large_data.loc[
    (large_data["model"]=="tiam-ucl")
    &(large_data["sector"]=="Electricity"),
    "val"
] /= 1000

large_data.loc[
    (large_data["model"].str.contains("remind"))
    &(large_data["sector"]=="Carbon Dioxide Removal"),
    "val"
] *= -1

large_data.loc[
    (large_data["model"] == "tiam-ucl")
    &(large_data["sector"]=="Carbon Dioxide Removal"),
    "val"
] /= 1000



In [18]:
large_data = large_data.loc[large_data["val"] != 0]
large_data = large_data.loc[large_data["val"].abs()>0.1]

In [19]:
fp = f"../data/structured_data {str(__version__)}.csv"
large_data.to_csv(fp)